In [2]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn import preprocessing

In [6]:
nor = preprocessing.MinMaxScaler()
train_path = r"D:\Datasets\house prices\train.csv"
test_path = r"D:\Datasets\house prices\test.csv"
train_file = pd.read_csv(train_path)
test_file = pd.read_csv(test_path)
train_file.drop("Id", axis=1, inplace=True)
test_file.drop("Id", axis=1, inplace=True)
train_file.fillna(train_file.median(), inplace=True)
test_file.fillna(train_file.median(), inplace=True)
y_train = train_file["SalePrice"].values
y_train.resize(1460, 1)
num_cols = list(train_file._get_numeric_data().columns)
num_cols.remove("SalePrice")
train_file = pd.get_dummies(train_file, drop_first=True, dummy_na=True)
test_file = pd.get_dummies(test_file, drop_first=True, dummy_na=True)
train_file = nor.fit_transform(train_file)
test_file = nor.fit_transform(test_file)
X_train = train_file
x1 = pd.DataFrame(train_file)
x2 = pd.DataFrame(test_file)
x2 = x2.align(x1, axis=1)[0]
X_test = x2.values

In [7]:
X_train = Variable(torch.from_numpy(X_train).float())
y_train = Variable(torch.from_numpy(y_train).float())
X_test = Variable(torch.from_numpy(X_test).float())

In [8]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(289, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 1)
        self.drop = nn.Dropout(0.2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop(F.relu(self.fc2(x)))
        x = self.drop(F.relu(self.fc3(x)))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        return x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = Model().cuda()

In [10]:
loss_fn = nn.MSELoss()
opt = torch.optim.Adam(net.parameters())

In [11]:
epochs = 10000
train_loss = 0
for epoch in range(epochs):
    preds = net(X_train.cuda())
    opt.zero_grad()
    loss = loss_fn(preds, y_train.cuda())
    loss.backward()
    opt.step()
    train_loss += loss.item()
    if epoch % 100 == 0:
        print(f"Loss: {loss.item()/(epoch+1)}")

Loss: 39039266816.0
Loss: 386527394.2178218
Loss: 194225208.039801
Loss: 129698560.85049833
Loss: 97354780.08977556
Loss: 77922688.25548902
Loss: 64957182.722129785
Loss: 55690822.84736092
Loss: 48738160.81897628
Loss: 43328819.99556049
Loss: 39000266.54945055
Loss: 35458008.00726612
Loss: 32505634.318068277
Loss: 30007122.840891622
Loss: 27865286.806566738
Loss: 26008838.651565623
Loss: 24384301.571517803
Loss: 22950774.142269254
Loss: 21676439.098278735
Loss: 20536174.022093635
Loss: 19509878.468765616
Loss: 18581278.82722513
Loss: 17737058.980463427
Loss: 16966217.65145589
Loss: 16259586.345689297
Loss: 15609462.941223511
Loss: 15009329.802383699
Loss: 14453634.511662347
Loss: 13937617.570867548
Loss: 13457175.738021372
Loss: 13008752.687770743
Loss: 12589250.827475008
Loss: 12195959.642611684
Loss: 11826497.066343533
Loss: 11478761.192590415
Loss: 11150890.264495859
Loss: 10841229.329630658
Loss: 10548302.300999729
Loss: 10270788.42830834
Loss: 10007502.388105614
Loss: 9757377.3596

In [18]:
pred = net(X_test.cuda())

In [19]:
pred[:20]

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], device='cuda:0', grad_fn=<SliceBackward>)